In [1]:
import yaml
import add_packages
from pprint import pprint
import os, re
import pandas as pd
# import tqdm
from tqdm.auto import tqdm

from toolkit.langchain import (
	document_loaders, text_splitters, text_embedding_models, stores, 
	prompts, utils, output_parsers, agents, documents, models,
	runnables, tools, chains
)

from toolkit import sql

PATH_DATA = f"{add_packages.APP_PATH}/data/vtc"
FILE_CFG = "vtc.yaml"
tqdm.pandas(desc="Processing")

with open(f"{add_packages.APP_PATH}/my_configs/{FILE_CFG}", 'r') as file:
    configs = yaml.safe_load(file)

/opt/homebrew/Caskroom/miniconda/base/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm = models.chat_openai
my_sql_db = sql.MySQLDatabase()
embeddings = text_embedding_models.OpenAIEmbeddings()
vectorstore = stores.faiss.FAISS

# Data

## txt - FAQ

### vtc_faq

In [3]:
path_txt = f"{PATH_DATA}/vtc_faq.txt"

In [4]:
loader_txt = document_loaders.TextLoader(path_txt)
doc_txt = loader_txt.load()

text_splitter = text_splitters.RecursiveCharacterTextSplitter(
	# chunk_size=500, chunk_overlap=100,
	separators=["##"], chunk_size=150, chunk_overlap=0,
)
docs_txt = text_splitter.split_documents(doc_txt)
docs_txt = docs_txt[1:]

metadatas = {
	"data": "frequently asked questions"
}
utils.remove_metadata(docs_txt, "source")
utils.update_metadata(docs_txt, metadatas)

In [5]:
docs_txt_vtc_faq = docs_txt

### onli_faq

In [3]:
path_txt = f"{PATH_DATA}/onli_faq.txt"

In [4]:
loader_txt = document_loaders.TextLoader(path_txt)
doc_txt = loader_txt.load()

text_splitter = text_splitters.RecursiveCharacterTextSplitter(
	# chunk_size=500, chunk_overlap=100,
	separators=["##"], chunk_size=150, chunk_overlap=0,
)
docs_txt = text_splitter.split_documents(doc_txt)
docs_txt = docs_txt[1:]

metadatas = {
	"data": "frequently asked questions"
}
utils.remove_metadata(docs_txt, "source")
utils.update_metadata(docs_txt, metadatas)

In [5]:
docs_txt_onli_faq = docs_txt

### File 3

In [ ]:
path_txt = f"{PATH_DATA}/faq.txt"

In [ ]:
loader_txt = document_loaders.TextLoader(path_txt)
doc_txt = loader_txt.load()

text_splitter = text_splitters.RecursiveCharacterTextSplitter(
	# chunk_size=500, chunk_overlap=100,
	separators=["##"], chunk_size=150, chunk_overlap=0,
)
docs_txt = text_splitter.split_documents(doc_txt)
docs_txt = docs_txt[1:]

metadatas = {
	"data": "frequently asked questions"
}
utils.remove_metadata(docs_txt, "source")
utils.update_metadata(docs_txt, metadatas)

## table

### onli_course_list

In [ ]:
file_csv = "onli_course_list.xlsx"
path_csv = f"{PATH_DATA}/{file_csv}"
path_csv_processed = f"{PATH_DATA}/{file_csv.split('.')[0]}2.xlsx"

In [ ]:
df = pd.read_excel(
	path_csv, 
 	# delimiter="," # "," ";"
)

df.head()

#### Process

In [ ]:
model = models.chat_openai

template1 = """\
Here is a description of a learning course:

<course_description>
{{COURSE_DESCRIPTION}}
</course_description>

Please read the course description carefully. Your task is to extract the core information from the description, leaving out any boilerplate or filler words that are not essential to understanding the key points of the course.

Summarize the core information you have extracted in a single short paragraph. 

Provide your output in Vietnamese.
{text}"""

template2 = """\
...
{text}
"""

prompt_template1 = prompts.PromptTemplate.from_template(template1)
prompt_template2 = prompts.PromptTemplate.from_template(template2)

chain1 = prompt_template1 | model | output_parsers.StrOutputParser()
chain2 = prompt_template2 | model | output_parsers.StrOutputParser()

# chain = runnables.RunnablePassthrough.assign(
#   text=chain1
# ).assign(
#   text=chain2
# )

chain = runnables.RunnablePassthrough.assign(
  text=chain1
)

def process_csv_col(text: str) -> str:
  result = chain.invoke({"text": text})['text']
  return result

# query = '...'
# result = process_csv_col(query)
# pprint(result)

In [ ]:
df = pd.read_csv(path_csv)

col_to_process = "MÔ TẢ (course description)"

df[col_to_process] = df[col_to_process].progress_apply(process_csv_col)

df.to_excel(f"{path_csv_processed}", index=False)


In [ ]:
df.to_excel(f"{path_csv_processed}", index=False)

In [ ]:
path_csv = path_csv_processed

#### Load

In [ ]:
path_csv

In [ ]:
csv_cols = list(df.columns)

loader_csv = document_loaders.UnstructuredExcelLoader(
	path_csv,
	mode="elements"
)
docs_csv = loader_csv.load_and_split()

In [ ]:
docs_csv = loader_csv.load()[1:]

metadatas = {
	"data": "..."
}

utils.remove_metadata(docs_csv, "source")
utils.remove_metadata(docs_csv, "row")
utils.update_metadata(docs_csv, metadatas)

#### Load to sql

In [3]:
my_table_schema = [
	"course_id SERIAL",
	"course_name VARCHAR(255) NOT NULL UNIQUE",
	"course_category VARCHAR(255) NOT NULL",
	"instructor_name VARCHAR(100) NOT NULL",
	"course_link VARCHAR(2048) NOT NULL UNIQUE",
	"course_description TEXT NOT NULL",
	"PRIMARY KEY (course_id)",
]
my_table = sql.MySQLTable(
	name="onli_course_list", 
	schema=my_table_schema,
	db=my_sql_db,
)
my_table.create()

db = stores.SQLDatabase.from_uri(my_sql_db.get_uri())
llm = models.chat_openai

embeddings = text_embedding_models.OpenAIEmbeddings()
vectorstore = stores.faiss.FAISS

table_cols = [col_description.split(" ")[0] for col_description in my_table_schema][1:-1]


In [ ]:
df = pd.read_excel(path_csv)
df.columns = table_cols

my_table.insert_from_dataframe(df)

In [4]:
cols = ["course_category", "instructor_name"]
proper_nouns_onli_course_list = [value for col in cols for value in my_table.get_discrete_values_col(col)]

In [ ]:
questions = [
  "Which courses are available in the Design category?",
	"Who are the instructors for the Personal Development courses?",
	"Can you provide a summary of the course descriptions for the Digital Marketing category?",
	"Which courses have the longest descriptions?",
	"How many courses are offered by each instructor?",
	"Which course categories have the most number of courses?",
	"Can you tell me about the course 'Kỹ năng quản lý thời gian'?",
	"Which courses have the shortest descriptions?",
	"Which instructors have the most number of courses?",
	"Can you list all the courses that are related to Personal Development?"
]

examples_questions_to_sql_onli_course_list = [
    {
        "input": "Which courses are available in the Design category?",
        "query": "SELECT course_name FROM courses WHERE course_category = 'Design';"
    },
    {
        "input": "Who are the instructors for the Personal Development courses?",
        "query": "SELECT DISTINCT instructor_name FROM courses WHERE course_category = 'Personal Development';"
    },
    {
        "input": "Can you provide a summary of the course descriptions for the Digital Marketing category?",
        "query": "SELECT course_name, SUBSTRING(course_description, 1, 100) AS summary FROM courses WHERE course_category = 'Digital Marketing';"
    },
    {
        "input": "Which courses have the longest descriptions?",
        "query": "SELECT course_name, LENGTH(course_description) AS description_length FROM courses ORDER BY description_length DESC LIMIT 5;"
    },
    {
        "input": "How many courses are offered by each instructor?",
        "query": "SELECT instructor_name, COUNT(course_id) AS num_courses FROM courses GROUP BY instructor_name;"
    },
    {
        "input": "Which course categories have the most number of courses?",
        "query": "SELECT course_category, COUNT(course_id) AS num_courses FROM courses GROUP BY course_category ORDER BY num_courses DESC LIMIT 5;"
    },
    {
        "input": "Can you tell me about the course 'Kỹ năng quản lý thời gian'?",
        "query": "SELECT * FROM courses WHERE course_name = 'Kỹ năng quản lý thời gian';"
    },
    {
        "input": "Which courses have the shortest descriptions?",
        "query": "SELECT course_name, LENGTH(course_description) AS description_length FROM courses ORDER BY description_length ASC LIMIT 5;"
    },
    {
        "input": "Which instructors have the most number of courses?",
        "query": "SELECT instructor_name, COUNT(course_id) AS num_courses FROM courses GROUP BY instructor_name ORDER BY num_courses DESC LIMIT 5;"
    },
    {
        "input": "Can you list all the courses that are related to Personal Development?",
        "query": "SELECT course_name FROM courses WHERE course_category = 'Personal Development';"
    }
]

### File vtc_course_list

In [ ]:
file_xlsx = "vtc_course_list.xlsx"
path_xlsx = f"{PATH_DATA}/{file_xlsx}"
path_xlsx_processed = f"{PATH_DATA}/{file_xlsx.split('.')[0]}-prod.xlsx"

In [ ]:
df = pd.read_excel(
	path_xlsx, 
 	# delimiter=";"
)

df.head()

#### Process

In [ ]:
model = models.chat_openai

template1 = """\
...
{text}"""

template2 = """\
...
{text}
"""

prompt_template1 = prompts.PromptTemplate.from_template(template1)
prompt_template2 = prompts.PromptTemplate.from_template(template2)

chain1 = prompt_template1 | model | output_parsers.StrOutputParser()
chain2 = prompt_template2 | model | output_parsers.StrOutputParser()

chain = runnables.RunnablePassthrough.assign(
  text=chain1
).assign(
  text=chain2
)

def process_xlsx_col(text: str) -> str:
  result = chain.invoke({"text": text})['text']
  return result

query = '...'
result = process_xlsx_col(query)

pprint(result)

In [ ]:
df = pd.read_excel(path_xlsx)

col_to_process = "..."

df[col_to_process] = df[col_to_process].progress_apply(process_xlsx_col)

df.to_excel(f"{path_xlsx_processed}", index=False)


In [ ]:
path_xlsx = path_xlsx_processed

#### Load

In [ ]:
xlsx_cols = ...

loader_xlsx = document_loaders.UnstructuredExcelLoader(
	path_xlsx,
	mode="elements",
)
docs_xlsx = loader_xlsx.load()

metadatas = {
	"data": "..."
}

utils.remove_metadata(docs_xlsx, "source")
utils.remove_metadata(docs_xlsx, "row")
utils.update_metadata(docs_xlsx, metadatas)

#### Load to sql

In [5]:
my_table_schema = [
	"course_id SERIAL",
	"course_name VARCHAR(255) NOT NULL",
	"semester_no INT NOT NULL",
	"semester_name VARCHAR(50) NOT NULL",
	"subject VARCHAR(100) NOT NULL",
	"hour INT NOT NULL",
	"learning_outcome TEXT NOT NULL",
	"PRIMARY KEY (course_id)",
]

my_table = sql.MySQLTable(
	name="vtc_course_list", 
	schema=my_table_schema,
	db=my_sql_db,
)
my_table.create()

db = stores.SQLDatabase.from_uri(my_sql_db.get_uri())
llm = models.chat_openai

embeddings = text_embedding_models.OpenAIEmbeddings()
vectorstore = stores.faiss.FAISS

table_cols = [col_description.split(" ")[0] for col_description in my_table_schema][1:-1]


In [ ]:
df = pd.read_excel(path_xlsx)
df.columns = table_cols

In [6]:
# my_table.insert_from_dataframe(df)

cols = ["course_name", "semester_name", "subject"]
proper_nouns_vtc_course_list = [value for col in cols for value in my_table.get_discrete_values_col(col)]

### FINAL SQL TABLE

In [14]:
examples_fewshot_tmp = dict(configs["sql"]["examples_questions_to_sql"]).values()
examples_questions_to_sql = [example for sublist in examples_fewshot_tmp for example in sublist]

proper_nouns = configs["sql"]["proper_nouns"]

my_sql_db = sql.MySQLDatabase()

cfg_sql_tool = configs["sql"]["tool"]

my_sql_chain = chains.MySqlChain(
	my_sql_db=my_sql_db,
	llm=llm,
	embeddings=embeddings,
	vectorstore=vectorstore,
	proper_nouns=proper_nouns,
	k_retriever_proper_nouns=4,
	examples_questions_to_sql=examples_questions_to_sql,
	k_few_shot_examples=5,
	is_debug=False,
	tool_name=cfg_sql_tool["name"],
	tool_description=cfg_sql_tool["description"],
	tool_metadata=cfg_sql_tool["metadata"],
	tool_tags=cfg_sql_tool["tags"],
)

tool_chain_sql = my_sql_chain.create_tool_chain_sql()

In [ ]:
result = my_sql_chain.invoke_chain(
	# "how many courses are there in Onlinica?"
	"các khoá học Onlinica về lập trình"
)

pprint(result)

In [ ]:
pprint(result['output'])

# Vector store 

Note:
- `tiktoken` >= 0.6.0

## txt

### vtc_faq

In [3]:
qdrant_txt_vtc_faq = stores.QdrantWrapper(
  qdrant_host=os.getenv("QDRANT_HOST"),
  qdrant_api_key=os.getenv("QDRANT_API_KEY"),
  configs=configs,
  **configs["vector_db"]["qdrant"]["vtc_faq"]
)

2024-06-04 11:23:03.741 | INFO     | toolkit.langchain.stores:__init__:113 - Found collection: `vtc_faq`.
2024-06-04 11:23:03.742 | INFO     | toolkit.langchain.stores:__init__:126 - `vtc_faq` - Embeddings: openai - {'model': 'text-embedding-3-large'}, 3072
2024-06-04 11:23:03.788 | INFO     | toolkit.langchain.stores:__init__:145 - `vtc_faq` - Retriever: Vectorstore


In [4]:
# qdrant_txt_vtc_faq.add_documents(docs_txt_vtc_faq)

### onli_faq

In [5]:
qdrant_txt_onli_faq = stores.QdrantWrapper(
  qdrant_host=os.getenv("QDRANT_HOST"),
  qdrant_api_key=os.getenv("QDRANT_API_KEY"),
  configs=configs,
  **configs["vector_db"]["qdrant"]["onli_faq"]
)

2024-06-04 11:23:04.602 | INFO     | toolkit.langchain.stores:__init__:113 - Found collection: `onli_faq`.
2024-06-04 11:23:04.603 | INFO     | toolkit.langchain.stores:__init__:126 - `onli_faq` - Embeddings: openai - {'model': 'text-embedding-3-large'}, 3072
2024-06-04 11:23:04.650 | INFO     | toolkit.langchain.stores:__init__:145 - `onli_faq` - Retriever: Vectorstore


In [8]:
# qdrant_txt_onli_faq.add_documents(docs_txt_onli_faq)

100%|██████████| 21/21 [00:19<00:00,  1.06it/s]


### File 3

In [ ]:
qdrant_txt = stores.QdrantWrapper(
  qdrant_host=os.getenv("QDRANT_HOST"),
  qdrant_api_key=os.getenv("QDRANT_API_KEY"),
  configs=configs,
  **configs["vector_db"]["qdrant"]["..."]
)

In [ ]:
# qdrant_txt.add_documents(docs_txt)

## table

### File 1

In [ ]:
qdrant_csv = stores.QdrantWrapper(
	qdrant_host=os.getenv("QDRANT_HOST"),
	qdrant_api_key=os.getenv("QDRANT_API_KEY"),
	configs=configs,
	**configs["vector_db"]["qdrant"]["..."]
)

In [ ]:
qdrant_csv.add_documents(docs_csv)

### File 2

In [ ]:
qdrant_csv = stores.QdrantWrapper(
	qdrant_host=os.getenv("QDRANT_HOST"),
	qdrant_api_key=os.getenv("QDRANT_API_KEY"),
	configs=configs,
	**configs["vector_db"]["qdrant"]["..."]
)

In [ ]:
qdrant_csv.add_documents(docs_csv)

# Test

In [ ]:
qdrant_lectures_content = stores.QdrantWrapper(
  qdrant_host=os.getenv("QDRANT_HOST"),
  qdrant_api_key=os.getenv("QDRANT_API_KEY"),
  configs=configs,
  **configs["vector_db"]["qdrant"]["lectures_content"],
)

In [6]:
examples_fewshot_tmp = dict(configs["sql"]["examples_questions_to_sql"]).values()
examples_questions_to_sql = [example for sublist in examples_fewshot_tmp for example in sublist]

proper_nouns = configs["sql"]["proper_nouns"]

my_sql_db = sql.MySQLDatabase()

cfg_sql_tool = configs["sql"]["tool"]

my_sql_chain = chains.MySqlChain(
	my_sql_db=my_sql_db,
	llm=llm,
	embeddings=embeddings,
	vectorstore=vectorstore,
	proper_nouns=proper_nouns,
	k_retriever_proper_nouns=4,
	examples_questions_to_sql=examples_questions_to_sql,
	k_few_shot_examples=5,
	is_debug=True,
	tool_name=cfg_sql_tool["name"],
	tool_description=cfg_sql_tool["description"],
	tool_metadata=cfg_sql_tool["metadata"],
	tool_tags=cfg_sql_tool["tags"],
)

tool_chain_sql = my_sql_chain.create_tool_chain_sql()

In [7]:
llm = models.chat_openai

tools = [
	qdrant_txt_vtc_faq.retriever_tool,
	qdrant_txt_onli_faq.retriever_tool,
	tool_chain_sql,
]

system_message_custom = configs["prompts"]["system_message_vtc"]
prompt = prompts.create_prompt_tool_calling_agent(system_message_custom)

agent = agents.MyStatelessAgent(
	llm=llm,
	tools=tools,
	prompt=prompt,
	agent_type=configs["agents"]["type"],
	agent_verbose=True,
)

2024-06-04 11:23:21.123 | INFO     | toolkit.langchain.agents:_create_agent:157 - Agent type: tool_calling


In [9]:
result = await agent.invoke_agent(
  # "Xin chào",
  "Các khoá học về lập trình trên Onlinica?",
  # "Chi tiết khoá học THIẾT KẾ CƠ SỞ DỮ LIỆU Onlinica",
	history_type="in_memory"
)

2024-06-04 11:23:37.370 | INFO     | toolkit.langchain.agents:__init__:90 - User Id: admin
2024-06-04 11:23:37.371 | INFO     | toolkit.langchain.agents:__init__:91 - Session Id: Mark Ramos-60a18a5b
2024-06-04 11:23:37.371 | INFO     | toolkit.langchain.agents:__init__:92 - History Type: in_memory




> Entering new AgentExecutor chain...

Invoking: `sql_executor` with `{'question': 'Các khoá học về lập trình trên Onlinica'}`




{'question': 'Các khoá học về lập trình trên Onlinica', 'table_names_to_use': ['onli_course_list'], 'proper_nouns': 'Lập trình\nGiảng viên Onlinica\nGiảng viên Onlinica (AI)\nMs. Đoàn Phương Trúc', 'examples': [{'input': "Can you tell me about the course 'Kỹ năng quản lý thời gian'?", 'query': "SELECT * FROM courses WHERE course_name = 'Kỹ năng quản lý thời gian';"}, {'input': 'What are the different courses offered?', 'query': 'SELECT DISTINCT course_name FROM courses;'}, {'input': 'What is the distribution of course hours across different subjects?', 'query': 'SELECT subject, SUM(hour) AS total_hours FROM courses GROUP BY subject;'}, {'input': 'Which subject has the most number of courses?', 'query': 'SELECT subject, COUNT(course_name) AS num_courses FROM courses GROUP BY subject ORDER BY num_courses DESC LIMIT 1;'}, {'input': 'Can you provide a breakdown of learning outcomes for each course?', 'query': 'SELECT course_name, learning_outcome FROM courses;'}], 'query': 'SELECT "course_

In [20]:
pprint(result)

('Here are some programming courses offered by Onlinica:\n'
 '1. KHÁM PHÁ NGÀNH GAME DEVELOPER - NHẬP MÔN\n'
 '2. LẬP TRÌNH PYTHON CĂN BẢN VỚI BIẾN VÀ VÒNG LẶP\n'
 '3. XÂY DỰNG MỘT GAME ĐƠN GIẢN BẰNG PYTHON\n'
 '4. THIẾT KẾ CƠ SỞ DỮ LIỆU\n'
 '5. NHẬP MÔN NGÀNH LẬP TRÌNH WEB FULL-STACK\n'
 '\n'
 'If you need more information about any specific course, feel free to ask!')


In [ ]:
input_message = questions[1]
# await agent.stream_conversable_agent(questions[2])
result = agent.invoke_agent(input_message)
# await agent.stream_agent(input_message)
pprint(result)